<a href="https://colab.research.google.com/github/ricardoruedas/ML/blob/main/%5B05%5D%20-%20Arboles%20de%20decision/Evaluaci%C3%B3n_y_optimizaci%C3%B3n_Corto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arboles de decisión: Evaluacion y optimizacion de modelos - Ejercicio 4: Evaluación_y_optimización_Corto.ipynb

Este notebook es un **I do**: todo resuelto y explicado paso a paso.

**BLOQUE 1 · Búsqueda de Hiperparámetros**

¿Qué es un hiperparámetro?

Son “mandos” externos que no aprende el modelo por sí mismo (p. ej. la profundidad máxima de un árbol, el C de una regresión logística o el número de vecinos en KNN).

Elegirlos bien puede subir mucho el rendimiento; elegirlos mal te lleva a sobreajuste o bajo rendimiento.

#1.1 Grid Search (búsqueda en rejilla)

Idea: probar todas las combinaciones posibles de una rejilla de valores.

✅ Simple y exhaustivo.

❌ Puede ser lento si hay muchos hiperparámetros/valores.

In [2]:
#Ejemplo (clasificación con Regresión Logística):
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


X, y = load_breast_cancer(return_X_y=True)


pipe = Pipeline([
("scaler", StandardScaler()),
("logreg", LogisticRegression(max_iter=500, solver="saga"))
])


# C controla la fuerza de la regularización (más pequeño = más regularización)
param_grid = {
"logreg__penalty": ["l1", "l2"],
"logreg__C": [0.01, 0.1, 1, 10, 100]
}


cv = GridSearchCV(pipe, param_grid, cv=5, scoring="f1", n_jobs=-1)
cv.fit(X, y)
print("Mejor combinación:", cv.best_params_)
print("Mejor F1 (CV):", cv.best_score_)

Mejor combinación: {'logreg__C': 1, 'logreg__penalty': 'l2'}
Mejor F1 (CV): 0.9847673174099155


#1.2 Random Search (búsqueda aleatoria)

Idea: en vez de probar todo, probamos muestras aleatorias del espacio de hiperparámetros.

✅ Mucho más rápido y a menudo consigue resultados similares a grid.

✅ Permite explorar rangos amplios.

❌ El resultado depende del azar (aunque con suficientes iteraciones suele ir bien).

In [3]:
#Ejemplo (Random Forest):
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


rf = RandomForestClassifier(random_state=42)


param_dist = {
"n_estimators": [100, 200, 300, 500],
"max_depth": [None, 5, 10, 15, 20],
"min_samples_split": [2, 5, 10],
"min_samples_leaf": [1, 2, 4],
"max_features": ["sqrt", "log2", None]
}


random_search = RandomizedSearchCV(
rf,
param_distributions=param_dist,
n_iter=30, # nº de combinaciones aleatorias a probar
cv=5,
scoring="f1",
n_jobs=-1,
random_state=42
)

random_search.fit(X, y)
print("Mejor combinación:", random_search.best_params_)
print("Mejor F1 (CV):", random_search.best_score_)

Mejor combinación: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20}
Mejor F1 (CV): 0.9722580019068872


#1.3 Optimización Bayesiana

Idea: en lugar de probar a ciegas, un modelo probabilístico aprende qué zonas del espacio de hiperparámetros son más prometedoras y decide inteligentemente qué probar después.

✅ Suele necesitar menos evaluaciones para encontrar buenos resultados.

✅ Ideal cuando entrenar un modelo es caro.

❌ Requiere librerías adicionales (p. ej. scikit-optimize u optuna).

In [4]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.3 MB/s eta 0:00:00


In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 4.8 MB/s eta 0:00:00


In [6]:
# === Optimización "bayesiana" súper rápida con Optuna (para dummies) ===
# Si NO tienes Optuna instalada, ejecuta en una celda previa:  !pip install optuna

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

X, y = load_breast_cancer(return_X_y=True)

# División rápida para ver desempeño final (holdout) y no sólo CV
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

try:
    import optuna

    # Objetivo: ajustar SOLO "C" (suficiente para enseñar el concepto y que sea veloz)
    def objective(trial):
        C = trial.suggest_float("C", 1e-3, 1e3, log=True)
        # Pipeline correcto para evitar fugas y que escale dentro de cada fold
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(C=C, penalty="l2", solver="lbfgs", max_iter=500, n_jobs=None))
        ])
        # CV muy rápido (3 folds) y métrica f1 (mejor que accuracy en general)
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        score = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1).mean()
        return score

    # Estudio express: 10 intentos, semilla fija para reproducibilidad
    study = optuna.create_study(direction="maximize", study_name="logreg_fast", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=10, timeout=None)  # sube n_trials si quieres más calidad

    print("Mejores hiperparámetros (rápidos):", study.best_params)
    print("Mejor F1 (CV 3-fold):", round(study.best_value, 4))

    # Entrena modelo final con el mejor C en TODO el train y evalúa en test
    best_C = study.best_params["C"]
    final_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(C=best_C, penalty="l2", solver="lbfgs", max_iter=500))
    ])
    final_pipe.fit(X_train, y_train)
    y_pred = final_pipe.predict(X_test)
    print("Accuracy (test):", round(accuracy_score(y_test, y_pred), 4))
    print("F1 (test):", round(f1_score(y_test, y_pred), 4))

except ModuleNotFoundError:
    # Fallback mínimo: sin Optuna no hacemos bayesiana; mostramos qué instalar.
    print("No tienes 'optuna' instalado. Para usar la optimización bayesiana rápida, ejecuta antes:")
    print("    !pip install optuna")
    print("Luego vuelve a ejecutar esta celda.")


[I 2025-10-08 18:55:17,854] A new study created in memory with name: logreg_fast
[I 2025-10-08 18:55:17,926] Trial 0 finished with value: 0.9780390924804788 and parameters: {'C': 0.1767016940294795}. Best is trial 0 with value: 0.9780390924804788.
[I 2025-10-08 18:55:17,976] Trial 1 finished with value: 0.9720653051089453 and parameters: {'C': 506.1576888752306}. Best is trial 0 with value: 0.9780390924804788.
[I 2025-10-08 18:55:18,016] Trial 2 finished with value: 0.9795336384557486 and parameters: {'C': 24.658329458549105}. Best is trial 2 with value: 0.9795336384557486.
[I 2025-10-08 18:55:18,055] Trial 3 finished with value: 0.9759093668929735 and parameters: {'C': 3.907967156822881}. Best is trial 2 with value: 0.9795336384557486.
[I 2025-10-08 18:55:18,093] Trial 4 finished with value: 0.9623847110185908 and parameters: {'C': 0.008632008168602538}. Best is trial 2 with value: 0.9795336384557486.
[I 2025-10-08 18:55:18,131] Trial 5 finished with value: 0.9623847110185908 and para

Mejores hiperparámetros (rápidos): {'C': 24.658329458549105}
Mejor F1 (CV 3-fold): 0.9795
Accuracy (test): 0.972
F1 (test): 0.978


**BLOQUE 2 · Regularización (evitar sobreajuste)**

La regularización añade una “penalización” para evitar que el modelo se complique demasiado. Imagina que pones una goma que impide que los pesos crezcan sin control.

#2.1 L2 (Ridge)

Penaliza la suma de cuadrados de los pesos → tiende a mantener todos los pesos pequeños.

Suele estabilizar el modelo y reducir la varianza.

In [7]:
#Ejemplo (Logistic Regression L2):
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


pipe_l2 = Pipeline([
("scaler", StandardScaler()),
("clf", LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=500))
])
cv_scores = cross_val_score(pipe_l2, X, y, cv=5, scoring="f1")
print("F1 medio (L2, C=1):", cv_scores.mean())

# Prueba distintos C (0.01, 0.1, 1, 10). Más pequeño = más regularización.

F1 medio (L2, C=1): 0.9847673174099155


#2.2 L1 (Lasso)

Penaliza la suma de valores absolutos de los pesos → muchos pesos acaban en cero → hace selección de variables.

In [8]:
# Ejemplo (Logistic Regression L1):
pipe_l1 = Pipeline([
("scaler", StandardScaler()),
("clf", LogisticRegression(penalty="l1", C=0.5, solver="saga", max_iter=1000))
])
cv_scores = cross_val_score(pipe_l1, X, y, cv=5, scoring="f1")
print("F1 medio (L1, C=0.5):", cv_scores.mean())

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


F1 medio (L1, C=0.5): 0.9778134340475815


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#2.3 Early Stopping (parada temprana)

Para el entrenamiento antes de que el modelo empiece a sobreajustar.

útil en modelos que entrenan por épocas iterativas.

In [9]:
# Ejemplo sencillo con SGDClassifier (scikit‑learn puro):
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


sgd = SGDClassifier(loss="log_loss", early_stopping=True, n_iter_no_change=5, random_state=42)
sgd.fit(X_train, y_train)
print("Épocas usadas:", sgd.n_iter_)
print("F1 validación:", f1_score(y_val, sgd.predict(X_val)))
# En librerías de boosting (XGBoost/LightGBM/CatBoost) se usa un conjunto de validación y se detiene cuando la métrica ya no mejora tras N rondas.

Épocas usadas: 7
F1 validación: 0.9219858156028369


**BLOQUE 3 · Validación (comprobar que generaliza)**

Meta: estimar cómo rendirá el modelo en datos nuevos.

#3.1 Hold‑out vs K‑Fold Cross‑Validation

Hold‑out: una sola división (train/test). Simple pero ruidoso.

K‑Fold CV: dividir en K pliegues; entrenar K veces y promediar.

Ejemplo K‑Fold:

In [10]:
#Ejemplo K‑Fold:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


clf = DecisionTreeClassifier(random_state=42)
scores = cross_val_score(clf, X, y, cv=5, scoring="f1")
print("F1 medio (CV=5):", scores.mean())

F1 medio (CV=5): 0.9327183029780179


#3.2 Stratified Cross‑Validation

En clasificación, mantener la proporción de clases en cada pliegue evita estimaciones sesgadas.

In [11]:
#Ejemplo (comparar KFold vs StratifiedKFold):
# Comparación KFold vs StratifiedKFold (distribución de clases por fold)
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np

# Datos de ejemplo (binario y desbalance moderado)
X, y = load_breast_cancer(return_X_y=True)

print(f"Proporción positiva global: {y.mean():.3f}\n")

kf  = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("KFold (no estratificado):")
for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    fold_pos = y[test_idx].mean()
    print(f"  Fold {i+1}: % positiva = {fold_pos:.3f}")

print("\nStratifiedKFold:")
for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    fold_pos = y[test_idx].mean()
    print(f"  Fold {i+1}: % positiva = {fold_pos:.3f}")


Proporción positiva global: 0.627

KFold (no estratificado):
  Fold 1: % positiva = 0.623
  Fold 2: % positiva = 0.675
  Fold 3: % positiva = 0.623
  Fold 4: % positiva = 0.623
  Fold 5: % positiva = 0.593

StratifiedKFold:
  Fold 1: % positiva = 0.623
  Fold 2: % positiva = 0.623
  Fold 3: % positiva = 0.632
  Fold 4: % positiva = 0.632
  Fold 5: % positiva = 0.628


#3.3 Ejemplo claro de overfitting

Un árbol profundo memoriza; uno podado generaliza mejor.

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


deep_tree = DecisionTreeClassifier(max_depth=None, random_state=42)
shallow_tree = DecisionTreeClassifier(max_depth=3, random_state=42)


train_scores_deep = cross_val_score(deep_tree, X, y, cv=5, scoring="f1")
train_scores_shallow = cross_val_score(shallow_tree, X, y, cv=5, scoring="f1")


print("Árbol profundo F1 (CV):", train_scores_deep.mean())
print("Árbol podado F1 (CV):", train_scores_shallow.mean())

#Si evalúas sólo en entrenamiento, el árbol profundo parecerá “perfecto”. Con CV se ve si realmente generaliza.

Árbol profundo F1 (CV): 0.9327183029780179
Árbol podado F1 (CV): 0.9354371606953062


**BLOQUE 4 · Métricas esenciales**

#4.1 Clasificación

Accuracy: porcentaje de aciertos (cuidado con datos desbalanceados).

Precision: de los que predije como positivos, ¿cuántos lo son realmente?

Recall (sensibilidad): de todos los positivos reales, ¿cuántos detecté?

F1: media armónica de precision y recall (equilibrio).

ROC‑AUC: capacidad de ordenar bien positivos por encima de negativos.

Matriz de confusión: tabla con TP, FP, FN, TN.

In [13]:
#Ejemplo completo de métricas:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
model = RandomForestClassifier(n_estimators=300, random_state=42)
model.fit(X_train, y_train)


proba = model.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)


print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))
print("F1:", f1_score(y_test, pred))
print("ROC-AUC:", roc_auc_score(y_test, proba))
print("Matriz de confusión:\n", confusion_matrix(y_test, pred))
print("\nReporte completo:\n", classification_report(y_test, pred))

#Tip: ajusta el umbral (no siempre 0.5). Un umbral más bajo sube el recall a costa de bajar la precision.

Accuracy: 0.958041958041958
Precision: 0.9565217391304348
Recall: 0.9777777777777777
F1: 0.967032967032967
ROC-AUC: 0.9948637316561845
Matriz de confusión:
 [[49  4]
 [ 2 88]]

Reporte completo:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94        53
           1       0.96      0.98      0.97        90

    accuracy                           0.96       143
   macro avg       0.96      0.95      0.95       143
weighted avg       0.96      0.96      0.96       143



#4.2 (Bonus) Métricas de Regresión

MAE (error absoluto medio): fácil de interpretar.

MSE / RMSE (cuadrático): penaliza grandes errores.

R²: proporción de varianza explicada.

In [14]:
# Ejemplo rápido (California Housing):

# Métricas de regresión: MAE, RMSE y R² (compatible con sklearn antiguos)
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Datos
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Modelo
reg = RandomForestRegressor(n_estimators=300, random_state=42)
reg.fit(X_train, y_train)
pred = reg.predict(X_test)

# MAE
mae = mean_absolute_error(y_test, pred)

# RMSE: compatible con todas las versiones
try:
    # Intento rápido (sklearn recientes)
    rmse = mean_squared_error(y_test, pred, squared=False)
except TypeError:
    # Fallback para sklearn antiguos
    rmse = np.sqrt(mean_squared_error(y_test, pred))

# R²
r2 = r2_score(y_test, pred)

print("MAE:", round(mae, 4))
print("RMSE:", round(rmse, 4))
print("R²:", round(r2, 4))


MAE: 0.3271
RMSE: 0.502
R²: 0.8096
